# Cleaning data in Python

## Data type constraints

Numeric data or other type?

In [92]:
import pandas as pd

ride_sharing = pd.read_csv('files/ride_sharing_original.csv')

print(ride_sharing.info())
print(ride_sharing['user_type'].describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25760 entries, 0 to 25759
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ride_id          25760 non-null  int64 
 1   duration         25760 non-null  object
 2   station_A_id     25760 non-null  int64 
 3   station_A_name   25760 non-null  object
 4   station_B_id     25760 non-null  int64 
 5   station_B_name   25760 non-null  object
 6   bike_id          25760 non-null  int64 
 7   user_type        25760 non-null  int64 
 8   user_birth_year  25760 non-null  int64 
 9   user_gender      25760 non-null  object
dtypes: int64(6), object(4)
memory usage: 2.0+ MB
None
count    25760.000000
mean         2.008385
std          0.704541
min          1.000000
25%          2.000000
50%          2.000000
75%          3.000000
max          3.000000
Name: user_type, dtype: float64


In [93]:
ride_sharing['duration'].value_counts()

7 minutes      2322
8 minutes      2234
9 minutes      2096
6 minutes      2091
5 minutes      1998
               ... 
100 minutes       1
137 minutes       1
155 minutes       1
193 minutes       1
891 minutes       1
Name: duration, Length: 172, dtype: int64

Duration is a string with "minutes" at the end.

We need to convert it to a numeric type.

In [94]:
# Strip duration of minutes
ride_sharing['duration_time'] = ride_sharing['duration'].str.strip('minutes')

# Convert duration to integer
ride_sharing['duration_time'] = ride_sharing['duration_time'].astype('int')

# Write an assert statement making sure of conversion
assert ride_sharing['duration_time'].dtype == 'int'

# Print formed columns and calculate average ride duration 
print(ride_sharing[['duration','duration_time']])
print(ride_sharing['duration_time'].mean())

         duration  duration_time
0      12 minutes             12
1      24 minutes             24
2       8 minutes              8
3       4 minutes              4
4      11 minutes             11
...           ...            ...
25755  11 minutes             11
25756  10 minutes             10
25757  14 minutes             14
25758  14 minutes             14
25759  29 minutes             29

[25760 rows x 2 columns]
11.389052795031056


## Data range constraints

**Tire size constraints**

Bicycle tire sizes are correctly loaded as a categorical value.

In [95]:
import pandas as pd

ride_sharing = pd.read_csv('files/ride_sharing.csv', dtype={'tire_size':'category'})

ride_sharing['tire_size'].dtype.name

'category'

Bicycle tire sizes could be either 26″, 27″ or 29″.

In [96]:
ride_sharing['tire_size'].value_counts()

27    8678
26    8628
29    8454
Name: tire_size, dtype: int64

The ride sharing provider decided to set the maximum tire size to be 27″.

You can make sure the tire_sizes column has the correct range by first converting it to an integer, then setting and testing the new upper limit of 27″ for tire sizes.

In [97]:
# Convert tire_sizes to integer
ride_sharing['tire_size'] = ride_sharing['tire_size'].astype('int')

# Set all values above 27 to 27
ride_sharing.loc[ride_sharing['tire_size'] > 27, 'tire_size'] = 27

# Reconvert tire_sizes back to categorical
ride_sharing['tire_size'] = ride_sharing['tire_size'].astype('category')

# Print tire size description
print(ride_sharing['tire_size'].dtype)

# Print tire size value counts
print(ride_sharing['tire_size'].value_counts())

category
27    17132
26     8628
Name: tire_size, dtype: int64


Let's introduce a random ride date column to the data, and a bug with dates set in the future.

In [98]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

import pandas as pd
import numpy as np

ride_sharing = pd.read_csv('files/ride_sharing_original.csv')

today = datetime.today()
today_last_year = today - relativedelta(years=1)
# Introducing a future date bug
today_next_month = today + relativedelta(months=1)

ride_sharing['ride_dt'] = np.random.choice(pd.date_range(today_last_year, today_next_month), size=ride_sharing.shape[0])
ride_sharing['ride_dt'].describe(datetime_is_numeric=True)

count                            25760
mean     2022-06-11 13:04:11.781916672
min         2021-11-27 10:23:28.924774
25%      2022-03-04 10:23:28.924773888
50%      2022-06-11 10:23:28.924773888
75%      2022-09-18 10:23:28.924773888
max         2022-12-27 10:23:28.924774
Name: ride_dt, dtype: object

Let's save the data to a new file.

In [99]:
ride_sharing.to_csv('files/ride_sharing_bugged.csv', index=False, date_format='%Y-%m-%d')

Reloading the dataframe, date is now a string

In [100]:
import datetime as dt

import pandas as pd
import numpy as np

ride_sharing = pd.read_csv('files/ride_sharing_bugged.csv')

ride_sharing['ride_dt'].describe()

count          25760
unique           396
top       2021-12-11
freq              94
Name: ride_dt, dtype: object

In [101]:
# Convert ride_date to date
ride_sharing['ride_dt'] = pd.to_datetime(ride_sharing['ride_dt']).dt.date

# Save today's date
today = dt.date.today()

# Set all in the future to today's date
ride_sharing.loc[ride_sharing['ride_dt'] > today, 'ride_dt'] = today

# Print maximum of ride_dt column
print(ride_sharing['ride_dt'].max())

2022-11-27


## Uniqueness constraints

### Duplicate values

* All columns have the same values

| first_name | last_name | address | height | weight |
| --- | --- | --- | --- | --- |
| Justin | Saddlemyer | Boulevard du Jardin Botanique 3, Bruxelles | 193 cm | 87 kg |
| Justin | Saddlemyer | Boulevard du Jardin Botanique 3, Bruxelles | 193 cm | 87 kg |

* Most columns have the same values

| first_name | last_name | address | height | weight |
| --- | --- | --- | --- | --- |
| Justin | Saddlemyer | Boulevard du Jardin Botanique 3, Bruxelles | 193 cm | 87 kg |
| Justin | Saddlemyer | Boulevard du Jardin Botanique 3, Bruxelles | **194 cm** | 87 kg |

### Finding duplicates

In [106]:
import pandas as pd

ride_sharing = pd.read_csv('files/ride_sharing.csv')

# Find duplicates
duplicates = ride_sharing.duplicated('ride_id', keep = False)

# Sort your duplicated rides
duplicated_rides = ride_sharing[duplicates].sort_values('ride_id')

# Print relevant columns of duplicated_rides
print(duplicated_rides[['ride_id','duration_time','user_birth_year']])

     ride_id  duration_time  user_birth_year
20        20             16             1998
37        20              2             1998
55        55             11             1999
76        55             11             2099
101      101             10             1990
126      101             10             1990


### Treating duplicates

* user_birth_year: set to the minimum value
* duration: set to the mean value

In [109]:
# Drop complete duplicates from ride_sharing
ride_dup = ride_sharing.drop_duplicates()

# Create statistics dictionary for aggregation function
statistics = {'user_birth_year': 'min', 'duration_time': 'mean'}

# Group by ride_id and compute new statistics
ride_unique = ride_dup.groupby('ride_id').agg(statistics).reset_index()

# Find duplicated values again
duplicates = ride_unique.duplicated(subset = 'ride_id', keep = False)
duplicated_rides = ride_unique[duplicates == True]

# Assert duplicates are processed
assert duplicated_rides.shape[0] == 0

## Membership constraints

### Categories and membership constraints

**Predefined finite set of categories**

| Type of data | Example values | Numeric representation |
| --- | --- | --- |
| Marriage Status | unmarried, married * | 0, 1 |
| Household Income Category | 0-20K, 20-40K, ... | 0, 1, ... |
| Loan Status | default, payed, no_loan | 0, 1, 2 |

<sub>* Marriage status can only be unmarried _or_ married</sub>

Generating random data for analysis

In [192]:
import csv
from faker import Faker
import datetime

def generate_data(records, headers):
    # Z+ doesn't exist, we're introducing a bug
    blood_types = ['A+', 'A-', 'B+', 'B-', 'AB+', 'AB-', 'O+', 'O-', 'Z+']
    marriage_status = ['married', 'unmarried' , 'Married', 'Unmarried', 'MARRIED', 'UNMARRIED', 'M', 'U', ' married', ' unmarried ']

    fake = Faker('pt_BR')

    with open("files/people.csv", 'wt') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=headers)
        writer.writeheader()
        for i in range(records):
            writer.writerow(
                {
                    "id" : i,
                    "prefix" : fake.prefix(),
                    "name": fake.name(),
                    "birth_date" : fake.date(pattern="%Y-%m-%d", end_datetime=datetime.date(2000, 1,1)),
                    "blood_type" : fake.random_element(elements=blood_types),
                    "marriage_status": fake.random_element(elements=marriage_status),
                    "household_income": fake.pyfloat(left_digits=6, right_digits=2, positive=True),
                    "phone_number" : fake.phone_number(),
                    "email": fake.email(),
                    "address" : fake.address(),
                    "zip_code" : fake.postcode(),
                    "city" : fake.city(),
                    "state" : fake.state(),
                    "country" : fake.country(),
                    "year": fake.year(),
                    "time": fake.time(),
                    "url": fake.url(),
                    "text": fake.word(),
                }
            )
    

records = 1000
headers = ["id", "prefix", "name", "birth_date", "blood_type", "marriage_status", "household_income", "phone_number", "email",
            "address", "zip_code", "city", "state", "country", "year", "time", "url", "text"]
generate_data(records, headers)

#### Loading data

In [139]:
import pandas as pd

people = pd.read_csv('files/people.csv', index_col='id')

people.head()

,prefix,name,birth_date,blood_type,marriage_status,phone_number,email,address,zip_code,city,state,country,year,time,url,text
id,,,,,,,,,,,,,,,,
0,Dr.,Felipe Cardoso,1990-02-14,Z+,UNMARRIED,+55 (011) 5054-4164,levi89@example.org,"Avenida de Pereira, 3\nEstoril\n20245-176 Nogu...",80108073,da Cruz do Norte,Distrito Federal,Trindade e Tobago,2022,23:44:52,https://nascimento.br/,numquam
1,Sr.,Sra. Gabrielly Gonçalves,1972-05-31,Z+,Unmarried,(031) 2302 6231,tvieira@example.net,Chácara Joaquim da Cunha\nLagoa\n34887442 Nasc...,32956-176,Oliveira,São Paulo,Ilhas Marshall,2001,00:33:57,https://cardoso.br/,reiciendis
2,Sr.,Thales Ramos,1982-01-09,O+,UNMARRIED,+55 84 8090-1714,teixeirarebeca@example.net,"Trecho Isabella Monteiro, 58\nSagrada Família\...",43804709,Moura,São Paulo,Cabo Verde,2008,18:27:05,http://gomes.net/,reprehenderit
3,Dr.,Gustavo Henrique Gomes,1991-07-14,A+,UNMARRIED,31 9265 4071,barrosfernando@example.org,"Largo Ramos, 24\nVila Ouro Minas\n66229-008 Mo...",79006-899,Moura de Teixeira,Distrito Federal,Guernsey,1996,11:09:45,https://da.org/,optio
4,Sra.,Pedro Costa,1989-03-01,O+,UNMARRIED,+55 (084) 9964 0372,mourajoao-gabriel@example.com,"Praça Cardoso, 93\nIpe\n97557988 Pereira do Oe...",01959-302,Melo,Mato Grosso,Singapura,2012,16:18:36,http://www.ferreira.br/,fugiat


In [140]:
people['blood_type'].value_counts()

AB-    135
AB+    114
B+     111
O+     109
B-     109
O-     107
A+     106
A-     105
Z+     104
Name: blood_type, dtype: int64

#### Finding inconsistent categories

In [141]:
# Correct blood types
blood_types = ['A+', 'A-', 'B+', 'B-', 'AB+', 'AB-', 'O+', 'O-']
categories = pd.DataFrame(blood_types)
categories.columns = ['blood_type']

inconsistent_categories = set(people['blood_type']).difference(categories['blood_type'])
print(inconsistent_categories)

{'Z+'}


#### Dropping inconsistent categories

In [142]:
inconsistent_categories = set(people['blood_type']).difference(categories['blood_type'])
inconsistent_rows = people['blood_type'].isin(inconsistent_categories)
inconsistent_data = people[inconsistent_rows]
# Drop inconsistent categories and get consistent data only
consistent_data = people[~inconsistent_rows]

Types of consistency errors
1. Value inconsistency
   * Inconsistent fields: `'married'`, `'Maried'`, `'UNMARRIED'`, `'not married'`...
   * _Trailing white spaces: `'_married'` , `' married '` ..;
2. Collapsing too many categories to few
   * Creating new groups: `0-20K`, `20-40K` categories ... from continuous household income data
   * Mapping groups to new ones: Mapping household income categories to 2 `'rich'` , `'poor'`
3. Making sure data is of type `category`

In [160]:
import pandas as pd

people = pd.read_csv('files/people.csv', index_col='id')

# Get marriage status column
marriage_status = people['marriage_status']
marriage_status.value_counts()

M              114
MARRIED        109
married        107
Unmarried      102
Married        101
U              101
 unmarried      96
 married        96
UNMARRIED       94
unmarried       80
Name: marriage_status, dtype: int64

#### Fixing inconsistent values

In [162]:
import pandas as pd

people = pd.read_csv('files/people.csv', index_col='id')

# Fix upper and lowercase
people['marriage_status'] = people['marriage_status'].str.lower()

# Fixing leading and trailing spaces
people['marriage_status'] = people['marriage_status'].str.strip()

# Fixing incorrect values
people.loc[people["marriage_status"] == "m", "marriage_status"] = "married"
people.loc[people["marriage_status"] == "u", "marriage_status"] = "unmarried"

people['marriage_status'].value_counts()

married      527
unmarried    473
Name: marriage_status, dtype: int64

#### Collapsing data into categories
Create categories out of data: income_group column from income column

In [193]:
import pandas as pd

people = pd.read_csv('files/people.csv', index_col='id')

# Using cut() - create category ranges and names
ranges = [0, 200000, 500000, np.inf]
group_names = ['0-20K', '20K-50K', '50K+']

# Create income group column
people['income_group'] = pd.cut(people['household_income'], bins=ranges, labels=group_names)
people['income_group'].value_counts()

50K+       486
20K-50K    313
0-20K      201
Name: income_group, dtype: int64

### Collapsing data into categories

**Map categories to fewer ones**: reducing categories in categorical column.

operating_system column is: `'Microsoft'`, `'MacOS'`, `'Linux'`, `'IOS'`, `'Android'`

operating_system column should become: `'DesktopOS'`, `'MobileOS'`

#### Airline data

Possible categories for airline data

In [112]:
import pandas as pd

categories = pd.read_csv('files/categories.csv', index_col=0)

categories

,cleanliness,safety,satisfaction
0,Clean,Neutral,Very satisfied
1,Average,Very safe,Neutral
2,Somewhat clean,Somewhat safe,Somewhat satisfied
3,Somewhat dirty,Very unsafe,Somewhat unsatisfied
4,Dirty,Somewhat unsafe,Very unsatisfied


Analyzing the data, we can see that the airline data has the following categories

In [163]:
import pandas as pd

airlines = pd.read_csv('files/airlines.csv', index_col=0)

print('Cleanliness: ', airlines['cleanliness'].unique(), "\n")
print('Safety: ', airlines['safety'].unique(), "\n")
print('Satisfaction: ', airlines['satisfaction'].unique(), "\n")

Cleanliness:  ['Clean' 'Average' 'Somewhat clean' 'Unacceptable' 'Somewhat dirty'
 'Dirty'] 

Safety:  ['Neutral' 'Very safe' 'Somewhat safe' 'Very unsafe' 'Somewhat unsafe'] 

Satisfaction:  ['Very satisfied' 'Neutral' 'Somewhat satisfied' 'Somewhat unsatisfied'
 'Very unsatisfied'] 

['Asia' 'Canada/Mexico' 'West US' 'East US' 'Midwest US' 'EAST US'
 'Middle East' 'Europe' 'eur' 'Central/South America'
 'Australia/New Zealand' 'middle east']
['Hub' 'Small' '    Hub' 'Medium' 'Large' 'Hub     ' '    Small'
 'Medium     ' '    Medium' 'Small     ' '    Large' 'Large     ']


#### Inconsistent categories

In [164]:
import pandas as pd

airlines = pd.read_csv('files/airlines.csv', index_col=0)

# Print unique values of both columns
print(airlines['dest_region'].unique())
print(airlines['dest_size'].unique())

['Asia' 'Canada/Mexico' 'West US' 'East US' 'Midwest US' 'EAST US'
 'Middle East' 'Europe' 'eur' 'Central/South America'
 'Australia/New Zealand' 'middle east']
['Hub' 'Small' '    Hub' 'Medium' 'Large' 'Hub     ' '    Small'
 'Medium     ' '    Medium' 'Small     ' '    Large' 'Large     ']


### Remapping categories into ranges

In [175]:
import pandas as pd
import numpy as np

airlines = pd.read_csv('files/airlines.csv', index_col=0)

# Create ranges for categories
label_ranges = [0, 60, 180, np.inf]
label_names = ['short', 'medium', 'long']

# Create wait_type column
airlines['wait_type'] = pd.cut(airlines['wait_min'], bins = label_ranges, 
                                labels = label_names)

# Create mappings and replace
mappings = {'Monday':'weekday', 'Tuesday':'weekday', 'Wednesday': 'weekday', 
            'Thursday': 'weekday', 'Friday': 'weekday', 
            'Saturday': 'weekend', 'Sunday': 'weekend'}

airlines['day_week'] = airlines['day'].replace(mappings)

print(airlines['wait_type'].value_counts())
print(airlines['day_week'].value_counts())

medium    1711
long       685
short       81
Name: wait_type, dtype: int64
weekday    2000
weekend     477
Name: day_week, dtype: int64
